Problem Statement

Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

path = r"C:\Module\build-modules\customerPersonalityAnalysis\marketing_campaign.csv"

df = pd.read_csv(path,sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [148]:
# df.info()

In [149]:
# df.describe()

In [150]:
df.duplicated().sum()

np.int64(0)

In [151]:
df = df.replace([np.inf, -np.inf], np.nan)

In [152]:
df.isna().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

In [153]:
df.dropna(inplace=True)
# df.isna().sum()

In [154]:
for x in df.columns.to_list():
    print(f"{x}: \n{df[x].sort_values().unique()}\n")

ID: 
[    0     1     9 ... 11187 11188 11191]

Year_Birth: 
[1893 1899 1900 1940 1941 1943 1944 1945 1946 1947 1948 1949 1950 1951
 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965
 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979
 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996]

Education: 
['2n Cycle' 'Basic' 'Graduation' 'Master' 'PhD']

Marital_Status: 
['Absurd' 'Alone' 'Divorced' 'Married' 'Single' 'Together' 'Widow' 'YOLO']

Income: 
[  1730.   2447.   3502. ... 160803. 162397. 666666.]

Kidhome: 
[0 1 2]

Teenhome: 
[0 1 2]

Dt_Customer: 
['01-01-2013' '01-01-2014' '01-02-2013' '01-02-2014' '01-03-2013'
 '01-03-2014' '01-04-2013' '01-04-2014' '01-05-2013' '01-05-2014'
 '01-06-2013' '01-07-2013' '01-08-2012' '01-08-2013' '01-09-2012'
 '01-09-2013' '01-10-2012' '01-10-2013' '01-11-2012' '01-11-2013'
 '01-12-2012' '01-12-2013' '02-01-2013' '02-01-2014' '02-02-2013'
 '02-02-2014' '02-03-2013' '02-03-2

In [155]:
non_target = df.columns.to_list()
non_target.remove('Response')

In [156]:
df.drop(["ID", "Z_CostContact", "Z_Revenue"], axis=1, inplace=True) 

In [157]:
cats = list(df.select_dtypes(include=['object', 'datetime64[ns]','bool']))
nums = list(df.select_dtypes(include='int64'))

print(f"Categorical : \n{cats}")
print(f"Numerical : \n{nums}")

Categorical : 
['Education', 'Marital_Status', 'Dt_Customer']
Numerical : 
['Year_Birth', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response']


In [158]:
Q1 = df[nums].quantile(0.25)
Q3 = df[nums].quantile(0.75)

IQR = Q3-Q1

mask = ~(((df[nums]<(Q1 - 1.5 * IQR)) | (df[nums] > (Q3 + 1.5 * IQR))).any(axis=1))

rows_before = df.shape[0]
print(f"Number of rows before removing outlier : {rows_before}")
df_clean = df[mask].reset_index(drop=True)
rows_after = df_clean.shape[0] 
print(f"Number of rows before removing outlier : {rows_after}")

Number of rows before removing outlier : 2216
Number of rows before removing outlier : 1187
